In [1]:
from jupyter_client import find_connection_file
connection_file = find_connection_file()
print(connection_file)

/home/yonatan/.local/share/jupyter/runtime/kernel-v2-17295hSMazz4NqQjy.json


In [2]:
# Constants for this project
import constants
from constants import *

import auxFunctions as af

import importlib

import os
import numpy as np
import pandas as pd

import papermill as pm
import shutil
shutil.copy('constants.py', 'export/')
shutil.copy('auxFunctions.py', 'export/')

# Change the current working directory
os.chdir("/home/yonatan/Documents/projects/vaccines/code")

In [3]:
print(dataset_day_dicts_for_adjFC)

[{'Dataset': 'GSE125921.SDY1529', 'Days': ['D0', 'D84']}, {'Dataset': 'GSE13699.SDY1289', 'Days': ['D0', 'D28']}, {'Dataset': 'GSE169159', 'Days': ['D0', 'D42']}, {'Dataset': 'GSE41080.SDY212', 'Days': ['HAI.D0', 'HAI.D28']}, {'Dataset': 'GSE45735.SDY224', 'Days': ['HAI.D21', 'HAI.D0']}, {'Dataset': 'GSE47353.SDY80', 'Days': ['D70.nAb', 'D0.nAb']}, {'Dataset': 'GSE48018.SDY1276', 'Days': ['nAb.D28', 'nAb.D0']}, {'Dataset': 'GSE48023.SDY1276', 'Days': ['HAI.D14', 'HAI.D0']}, {'Dataset': 'GSE52245.SDY1260', 'Days': ['D0', 'D30']}, {'Dataset': 'GSE59635.SDY63', 'Days': ['HAI.D0', 'HAI.D28']}, {'Dataset': 'GSE59654.SDY180', 'Days': ['HAI.D28', 'HAI.D0']}, {'Dataset': 'GSE59654.SDY404', 'Days': ['HAI.D28', 'HAI.D0']}, {'Dataset': 'GSE59654.SDY520', 'Days': ['HAI.D28', 'HAI.D0']}, {'Dataset': 'GSE59743.SDY400', 'Days': ['HAI.D28', 'HAI.D0']}, {'Dataset': 'GSE79396.SDY984', 'Days': ['D0', 'D28']}, {'Dataset': 'SDY1325', 'Days': ['D0', 'D28']}, {'Dataset': 'SDY296', 'Days': ['D0.HAI', 'D28.HAI

In [4]:
# Read in Data and drop missing values
importlib.reload(af)
data_dir = af.get_dir_by_name("data")
df = pd.read_csv(os.path.join(data_dir, "../data/all_vaccines.csv"))
datasets = df.dropna(subset=[immage_col, age_col, dataset_col, uid_col, day_col, response_col])
dataset_names = datasets[dataset_col].unique()
# Get the info for all influenza datasets, excluding some.
influenza_df = pd.DataFrame(influenza_dicts)
all_sets_df = pd.DataFrame(dataset_day_dicts_for_adjFC)

# We can choose to analyze all datasets, only influenza ones and only non-influenza ones
if bInfluenza:
    # Get the info for all influenza datasets, excluding some.
    metadata = influenza_df
    dataset_names = metadata[dataset_col].unique().astype(str)
    dataset_names = list(set(dataset_names) - set(exclude_datasets))
elif bNonInfluenza:
    metadata = all_sets_df
    dataset_names = all_sets_df[dataset_col].unique().astype(str)
    dataset_names = list(set(dataset_names) - set(influenza_df[dataset_col]) - set(exclude_datasets))
else:
    metadata = all_sets_df
    dataset_names = all_sets_df[dataset_col].unique().astype(str)
    dataset_names = list(set(dataset_names) - set(exclude_datasets))


dataset_names = ["GSE48023.SDY1276"]
# Filter datasets and metadata according to the list of datasets we want to look at.
datasets = datasets.loc[datasets["Dataset"].isin(dataset_names)]
metadata =  metadata.loc[metadata["Dataset"].isin(dataset_names)]

age_str = f", subjects over the age of {age_threshlod}" if bOlderOnly else ""
print(f"""Analysis parameters: discarding serprotected subjects: {bDiscardSeroprotected}{age_str}""")

# Turn on debug here for running a single dataset through analyze_dataset()
bDebug = False
if bDebug:
    # Narrow to a specific datset
    dataset_name = "GSE48023.SDY1276"
    # Filter data
    name_mask = datasets[dataset_col] == dataset_name
    dataset = datasets.loc[name_mask].reset_index(drop=True)

    # Filter metadata
    name_mask = metadata[dataset_col] == dataset_name
    metadata = metadata.loc[name_mask].reset_index(drop=True)

    day = [x for x in metadata["Days"].iloc[0] if "D0" not in x][0]
    day0 = [x for x in metadata["Days"].iloc[0] if "D0" in x][0]

    strains = dataset.loc[dataset[day_col] == day][strain_col].unique()
    print(strains)
    if len(strains) > 1:
        # "Influenza" denotes an MFC calculation in the original dataset
        strains = list(set(strains) - set(["Influenza"])) 
        # Sort to maintain consistency (sorts in-place)
        strains.sort() 
    if len(strains) < 1:
        strains = ['single strain - missing data']

    strain_index = 0
    P = {
        "bAdjustMFC": bAdjustMFC,
        "dataset_name": dataset_name,
        "strain_index": strain_index,
        "day":  day,
        "day0":  day0,
        "strain": strains[strain_index],
        "strains": strains,
        "bPlotOnly": False,
        "bPlotThreshold": False,
    }
    af.analyze_dataset(dataset, P)

else:
    results = af.analyze_all_datasets(datasets, metadata)

Analysis parameters: discarding serprotected subjects: True
GSE48023.SDY1276
['A/Brisbane/10/2007', 'A/Brisbane/59/2007', 'B/Florida/4/2006']
Preprocessing dataset, computing adjusted FC


/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Preprocessing dataset, computing adjusted FC
Preprocessing dataset, computing adjusted FC


/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/yonatan/miniforge3/envs/vaccines/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [5]:
results

,Dataset,Strain,strain_index,Day,bAdjustMFC,"(F score, IMMAGE)","(F score, Age)","(F score, Multivariate)","(NR rate over threshold, IMMAGE)","(NR rate over threshold, Age)","(NR rate over threshold, Multivariate)","(NR rate under threshold, IMMAGE)","(NR rate under threshold, Age)","(NR rate under threshold, Multivariate)","(Composite, IMMAGE)","(Composite, Age)","(Composite, Multivariate)"
0,GSE48023.SDY1276,A_Brisbane_10_2007,0,HAI.D14,True,0.52,0.43,0.49,0.56,0.35,0.54,0.24,0.22,0.24,0.54,0.39,0.51
1,GSE48023.SDY1276,A_Brisbane_10_2007,0,HAI.D14,False,0.57,0.45,0.57,0.55,0.37,0.52,0.23,0.29,0.19,0.56,0.41,0.54
2,GSE48023.SDY1276,A_Brisbane_59_2007,1,HAI.D14,True,0.53,0.47,0.47,0.20,0.20,0.39,0.44,0.37,0.23,0.36,0.34,0.43
3,GSE48023.SDY1276,A_Brisbane_59_2007,1,HAI.D14,False,0.47,0.49,0.50,0.14,0.41,0.41,0.37,0.22,0.17,0.31,0.45,0.45
4,GSE48023.SDY1276,B_Florida_4_2006,2,HAI.D14,True,0.48,0.54,0.53,0.38,0.30,0.51,0.29,0.48,0.27,0.43,0.42,0.52
5,GSE48023.SDY1276,B_Florida_4_2006,2,HAI.D14,False,0.45,0.44,0.44,0.22,0.00,0.35,0.36,0.35,0.00,0.33,0.22,0.40


In [6]:
# Get all the analyses that look somewhat promising based on the composite (F1 and over-threshold rate) metric
score_mask = (results["Composite", "IMMAGE"] >  results["Composite", "Age"]) | (results["Composite", "Multivariate"] >  results["Composite", "Age"])
cols_to_access = [
    dataset_col,
    strain_col,
    strain_index_col,
    day_col,
    "max_difference"
]

# Dynamically add all sub-columns for 'Composite'
composite_columns = [col for col in results.columns if col[0] == 'Composite']
cols_to_access.extend(composite_columns)
results["max_difference"] = results.apply(lambda row: max(row["Composite", "IMMAGE"] - row["Composite", "Age"], row["Composite", "Multivariate"] - row["Composite", "Age"]), axis=1)
results = results.loc[score_mask]
results = results.sort_values(by="max_difference", ascending=False)
results = results.loc[results["max_difference"] > 0.1]

In [7]:
results

,Dataset,Strain,strain_index,Day,bAdjustMFC,"(F score, IMMAGE)","(F score, Age)","(F score, Multivariate)","(NR rate over threshold, IMMAGE)","(NR rate over threshold, Age)","(NR rate over threshold, Multivariate)","(NR rate under threshold, IMMAGE)","(NR rate under threshold, Age)","(NR rate under threshold, Multivariate)","(Composite, IMMAGE)","(Composite, Age)","(Composite, Multivariate)",max_difference
5,GSE48023.SDY1276,B_Florida_4_2006,2,HAI.D14,False,0.45,0.44,0.44,0.22,0.00,0.35,0.36,0.35,0.00,0.33,0.22,0.40,0.17
1,GSE48023.SDY1276,A_Brisbane_10_2007,0,HAI.D14,False,0.57,0.45,0.57,0.55,0.37,0.52,0.23,0.29,0.19,0.56,0.41,0.54,0.15
0,GSE48023.SDY1276,A_Brisbane_10_2007,0,HAI.D14,True,0.52,0.43,0.49,0.56,0.35,0.54,0.24,0.22,0.24,0.54,0.39,0.51,0.15
4,GSE48023.SDY1276,B_Florida_4_2006,2,HAI.D14,True,0.48,0.54,0.53,0.38,0.30,0.51,0.29,0.48,0.27,0.43,0.42,0.52,0.10


In [8]:
# Generate figures for promising datasets
for index, row in results.iterrows():
    parameters = {
        "bAdjustMFC" : row["bAdjustMFC"],
        "dataset_name": row[dataset_col],
        "strain_index": row[strain_index_col],
        "day": row[day_col]
    }
    adjFC_str = f"_adjFC" if row["bAdjustMFC"] else ""
    print(f'exporting {row[dataset_col]}, strain no. {row[strain_index_col]}: {row[strain_col]}, day: {row[day_col]}')
    output_notebook_name = f"{row[dataset_col]}_{row[strain_col]}_{row[day_col]}{seroprotected_str}{age_restrict_str}{adjFC_str}"
    output_notebook = f"export/{output_notebook_name}.ipynb"
    try:
            pm.execute_notebook(
                    input_path="vaccines-4.ipynb",
                    output_path=output_notebook,
                    parameters=parameters,
                    prepare_only=True
            )
    except Exception as e:
            print (f"******\nCaught exception when runnnig {output_notebook}\n******\n")
            raise(e)
    # Export the executed notebook to HTML
    output_html = f"{output_notebook_name}.html"
    os.system(f"jupyter nbconvert --execute --no-input --to html {output_notebook} --output {output_html}")

exporting GSE48023.SDY1276, strain no. 2: B_Florida_4_2006, day: HAI.D14


[NbConvertApp] Converting notebook export/GSE48023.SDY1276_B_Florida_4_2006_HAI.D14_discard_seroprotected.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 3 image(s).
[NbConvertApp] Writing 593708 bytes to export/GSE48023.SDY1276_B_Florida_4_2006_HAI.D14_discard_seroprotected.html


exporting GSE48023.SDY1276, strain no. 0: A_Brisbane_10_2007, day: HAI.D14


[NbConvertApp] Converting notebook export/GSE48023.SDY1276_A_Brisbane_10_2007_HAI.D14_discard_seroprotected.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 3 image(s).
[NbConvertApp] Writing 593174 bytes to export/GSE48023.SDY1276_A_Brisbane_10_2007_HAI.D14_discard_seroprotected.html


exporting GSE48023.SDY1276, strain no. 0: A_Brisbane_10_2007, day: HAI.D14


[NbConvertApp] Converting notebook export/GSE48023.SDY1276_A_Brisbane_10_2007_HAI.D14_discard_seroprotected_adjFC.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 3 image(s).
[NbConvertApp] Writing 592947 bytes to export/GSE48023.SDY1276_A_Brisbane_10_2007_HAI.D14_discard_seroprotected_adjFC.html


exporting GSE48023.SDY1276, strain no. 2: B_Florida_4_2006, day: HAI.D14


[NbConvertApp] Converting notebook export/GSE48023.SDY1276_B_Florida_4_2006_HAI.D14_discard_seroprotected_adjFC.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 3 image(s).
[NbConvertApp] Writing 585905 bytes to export/GSE48023.SDY1276_B_Florida_4_2006_HAI.D14_discard_seroprotected_adjFC.html


In [9]:
# Save promising results to a spreadsheet
def generate_html_path(row):
    output_html = f"{row[dataset_col]}_{row[strain_col]}_{row[day_col]}{seroprotected_str}{age_restrict_str}.html"
    output_html = os.path.join(af.get_dir_by_name('code'), "export", output_html)    
    output_html = '=HYPERLINK("file://' + output_html + '", "Link")'
    return output_html

results = results[cols_to_access] 
results['html_path'] = results.apply(generate_html_path, axis=1)
results.to_csv(f"export/results_{seroprotected_str}{age_restrict_str}{adjFC_str}.csv", index=False, float_format='%.2f')

In [10]:
results

,Dataset,Strain,strain_index,Day,max_difference,"(Composite, IMMAGE)","(Composite, Age)","(Composite, Multivariate)",html_path
5,GSE48023.SDY1276,B_Florida_4_2006,2,HAI.D14,0.17,0.33,0.22,0.40,"=HYPERLINK(""file:///home/yonatan/Documents/projects/vaccines/code/export/GSE48023.SDY1276_B_Florida_4_2006_HAI.D14_discard_seroprotected.html"", ""Link"")"
1,GSE48023.SDY1276,A_Brisbane_10_2007,0,HAI.D14,0.15,0.56,0.41,0.54,"=HYPERLINK(""file:///home/yonatan/Documents/projects/vaccines/code/export/GSE48023.SDY1276_A_Brisbane_10_2007_HAI.D14_discard_seroprotected.html"", ""Link"")"
0,GSE48023.SDY1276,A_Brisbane_10_2007,0,HAI.D14,0.15,0.54,0.39,0.51,"=HYPERLINK(""file:///home/yonatan/Documents/projects/vaccines/code/export/GSE48023.SDY1276_A_Brisbane_10_2007_HAI.D14_discard_seroprotected.html"", ""Link"")"
4,GSE48023.SDY1276,B_Florida_4_2006,2,HAI.D14,0.10,0.43,0.42,0.52,"=HYPERLINK(""file:///home/yonatan/Documents/projects/vaccines/code/export/GSE48023.SDY1276_B_Florida_4_2006_HAI.D14_discard_seroprotected.html"", ""Link"")"
